In [ ]:
!pip install torch -U
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
import transformers
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from transformers import EarlyStoppingCallback

import accelerate
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

### GPU 설정하기,

device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print("device : " , device)

device :  cuda:0


## 토크나이저 및 모델 불러오기

In [ ]:
Model_name = 'beomi/KcELECTRA-base-v2022'
tokenizer = AutoTokenizer.from_pretrained(Model_name)

## 데이터 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data1 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/word_out_언감O.xlsx", index_col = 0)
data2 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/word_in_언감O.xlsx", index_col = 0)
data3 = pd.read_excel("/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 판단 유무/data/wrong.xlsx", index_col = 0)

In [ ]:
data = pd.concat([data1, data2, data3], axis = 0)

In [ ]:
data.index = range(len(data))

In [ ]:
data[['장애및병력','성과가족','사회적신분','출신','욕설','기타','언어감수성_O']]

,장애및병력,성과가족,사회적신분,출신,욕설,기타,언어감수성_O
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
420970,0,0,0,1,0,0,0
420971,1,0,0,0,0,0,0
420972,0,1,0,0,0,0,0
420973,0,1,0,0,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data[['장애및병력','성과가족','사회적신분','출신','욕설','기타','언어감수성_O']], test_size=0.2, shuffle = True)

In [ ]:
tokenized_train_sentence = tokenizer(
    list(X_train),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

In [ ]:
print(tokenized_train_sentence[0].tokens)

['[CLS]', '개', '##모', '##차', '산', '[UNK]', '있니', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]

In [ ]:
tokenized_test_sentence = tokenizer(
    list(X_test),
    return_tensors = 'pt',
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True)

### 이젠 데이터를 torch 버전으로 변환해주기

In [ ]:
class CurseDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

### model 학습

In [ ]:
train_label = y_train.values
test_label = y_test.values

train_dataset = CurseDataset(tokenized_train_sentence, train_label)
test_dataset = CurseDataset(tokenized_test_sentence, test_label)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(Model_name, num_labels = 7)
model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:

import accelerate

In [ ]:
training = TrainingArguments(
    output_dir='./',
    num_train_epochs = 40,
    per_device_train_batch_size=224,
    per_device_eval_batch_size=64,
    logging_dir='./logs',
    logging_steps=500,
    save_total_limit=2,
    save_strategy = 'steps',
    evaluation_strategy="steps",  # 이를 "steps" 또는 "epoch"으로 설정하여 정기적으로 평가를 수행
    eval_steps=500,  # 평가를 얼마나 자주 수행할지 (500 스텝마다 평가)
    load_best_model_at_end=True,  # 학습이 끝날 때 최고의 모델을 로드
    metric_for_best_model="f1",  # 모델의 성능을 비교하기 위한 메트릭 설정 (여기서는 f1 점수)
    greater_is_better=True,  # f1 점수가 클수록 좋으므로 True로 설정
      # 개선이 없을 때 얼마나 많은 평가를 기다릴 것인지
)


In [ ]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision , recall, f1, _ = precision_recall_fscore_support(labels, preds, average = 'binary')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy ' : acc,
      'f1': f1,
      'precision' : precision,
      'recall' : recall
  }

In [ ]:
trainer = Trainer(
    model = model,
    args = training,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-13-d3da3f7a2d94>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


OutOfMemoryError: ignored

In [ ]:
trainer.evaluate(eval_dataset = test_dataset)

In [ ]:
def predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
      sent,
      return_tensors = 'pt',
      truncation = True,
      add_special_tokens = True,
      max_length = 128
  )

  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent['input_ids'],
        attention_mask = tokenized_sent['attention_mask'],
        token_type_ids = tokenized_sent['token_type_ids']
    )

  logits = outputs[0]
  logits = logits.detach().cpu()
  result = logits.argmax(-1)

  if result == 0:
    result = '언어감수성이 낮습니다'
  elif result == 1:
    result = '언어감수성이 높습니다'
  return result

In [ ]:
predict("너 오른손잡이야?")

NameError: ignored

In [ ]:
model.save_pretrained('directory_path')
tokenizer.save_pretrained('directory_path')

('directory_path/tokenizer_config.json',
 'directory_path/special_tokens_map.json',
 'directory_path/tokenizer.json')

In [ ]:
from google.colab import files

# 저장된 폴더를 zip 파일로 압축
!zip -r /content/directory_path.zip /content/directory_path

In [ ]:
# 모델 저장 경로 설정
save_path = "/content/drive/MyDrive/데이터 청년 캠퍼스/모델링/언어감수성 카테고리 판단 유무/Result"

# 모델과 토크나이저 저장
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
